In [0]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import model_selection, linear_model, preprocessing, ensemble, metrics, svm
from sklearn.model_selection import cross_val_score
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.utils import shuffle
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt', 'shuffle']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [0]:
train_data = pd.read_csv('train 2.csv', sep =',')

In [0]:
print (train_data.shape)
train_data.head()

(30000, 231)


,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,Var11,Var12,Var13,Var14,Var15,Var16,Var17,Var18,Var19,Var20,Var21,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,Var34,Var35,Var36,Var37,Var38,Var39,Var40,...,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,Var201,Var202,Var203,Var204,Var205,Var206,Var207,Var208,Var209,Var210,Var211,Var212,Var213,Var214,Var215,Var216,Var217,Var218,Var219,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,labels
0,NaN,NaN,NaN,NaN,NaN,1176.0,7.0,NaN,NaN,NaN,NaN,NaN,1652.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,264.0,330.0,NaN,4.0,32.0,NaN,NaN,356.56,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1670778.0,NaN,NaN,...,a4vvyxLkBI,RO12,NaN,taul,1K8T,TIPl,8Nsn57c,y4g9XoZ,NaN,NaN,rF6a,9_Y1,vzJD,VpdQ,zm5i,me75fM6ugJ,kIsH,NaN,uKAI,L84s,NhsEn4L,NaN,NaN,NaN,kZJtVhC,YcYG,UYBR,FzaX,07m8iqZ,oslk,VjJAgOW,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
1,NaN,NaN,NaN,NaN,NaN,1813.0,14.0,NaN,NaN,NaN,NaN,NaN,524.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128.0,160.0,NaN,2.0,24.0,NaN,NaN,200.00,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,3324282.0,NaN,NaN,...,wRXmfo875g,LrdZy8QqgUfkVShG,NaN,taul,1K8T,LloI,fhk21Ss,9YuuPSz3BI,qVbt9BD,NaN,VNo6,9_Y1,gnEy,VpdQ,6JmL,7M47J5GA0pTYIFxg5uy,kIsH,NaN,uKAI,L84s,4kVnq_T26xq1p,NaN,Dl9TQkS,NaN,jaw2yIk,fpq1,cJvF,FzaX,4UxGlow,zCkv,catzS2D,LM8l689qOp,NaN,ELof,3Cy4,ZI9m,ib5G6X1eUxUn6,mj86,NaN,-1
2,NaN,NaN,NaN,NaN,NaN,777.0,7.0,NaN,NaN,NaN,NaN,NaN,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128.0,160.0,NaN,2.0,56.0,NaN,NaN,186.64,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,6447600.0,NaN,NaN,...,DHeTmBftjz,RO12,NaN,taul,1K8T,LFM_,DhOMuhr,HKFGqS8,NaN,NaN,qljo,9_Y1,4N0K,sJzTlal,zm5i,me75fM6ugJ,kIsH,NaN,uKAI,L84s,NhsEn4L,NaN,NaN,NaN,mAjbk_S,EPl3,cJvF,FzaX,3aBuLpj,oslk,jjOYyXX,LM8l689qOp,NaN,NaN,Qu4f,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
3,NaN,NaN,NaN,NaN,NaN,1554.0,7.0,NaN,NaN,NaN,NaN,NaN,72.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,444.0,555.0,NaN,8.0,368.0,NaN,NaN,166.56,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,39258.0,NaN,NaN,...,avkq9ayfAo,RO12,NaN,taul,1K8T,LZs_,HzRXa4U,V_KvNzO,G8rYZbN,NaN,6v5P,9_Y1,000J,NaN,IYzP,DHn_WUyBhW_whjA88g9bvA64_,kIsH,NaN,uKAI,L84s,CrNX,NaN,dRv1k9E,NaN,mAjbq_f,b9MD,cJvF,OFWH,FMZBCuo,d0EEeJi,MHP8mHO,LM8l689qOp,NaN,ELof,7P5s,nIGXDli,F2FyR07IdsN7I,mj86,NaN,1
4,NaN,NaN,NaN,NaN,NaN,980.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,336.0,420.0,NaN,2.0,128.0,NaN,NaN,200.00,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1418100.0,NaN,NaN,...,CxSr4RXktW,AERks4l,NaN,taul,1K8T,uErj,ON5Ou9e,FoJylxy,NaN,NaN,F4f_,9_Y1,Albh,VpdQ,IYzP,DHn_WUyBhW_whjA88g9bvA64_,kIsH,NaN,uKAI,L84s,JBfYVit4g8,NaN,NaN,NaN,TDc_9Yi,Y5id,cJvF,FzaX,p40KIAf,Al6ZaUT,C6OGkwT,LM8l689qOp,NaN,NaN,Xa3G,02N6s8f,xwM2aC7IdeMC0,NaN,NaN,-1


In [0]:
from sklearn.utils import shuffle
# так как классы не сбалансированы проводим oversampling, добавляем необходимое количество строк с классом "отток" так, 
# чтобы была пропорция классов примерно 1:1
def oversampling(data):
    dataChurn = data[data['labels'] == 1]
    dataNoChurn = data[data['labels'] == -1]
    alpha = (int)(dataNoChurn.shape[0]/dataChurn.shape[0])
    dataChurnRet = dataChurn 
    for i in range(alpha):
        dataChurnRet = dataChurnRet.append(dataChurn)
    dataNoChurn = dataNoChurn.append(dataChurnRet)
    dataNoChurn = shuffle(dataNoChurn)
    return dataNoChurn.iloc[:,:230], dataNoChurn.iloc[:,230:]
# ради эксперимента попробовал undersampling. Результаты не намного хуже, чем с oversampling'ом
def undersampling(data):
    dataChurn = data[data['labels'] == 1]
    dataNoChurn = data[data['labels'] == -1]
    countFirst = len(dataChurn)
    dataChurn = dataChurn.append(dataNoChurn.iloc[:countFirst,:])
    dataChurn = shuffle(dataChurn)
    return dataChurn.iloc[:,:230], dataChurn.iloc[:,230:]

# 1. Преобразование данных:

In [0]:
#заменяем все пропуски в датасете на нули:
train_data = train_data.fillna(0)
#проводим undersampling для того, чтобы сделать пропорцию классов 1:1 :
x_data, y_data = undersampling(train_data)

In [0]:
le = preprocessing.LabelEncoder()

# Encode first DataFrame 2 (where there is a combination of floats and strings)
x_data = x_data.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')
print("x_data encoded")

x_data encoded


# 2. Построение моделей и оценка качества кросс-валидации:

In [0]:
# печатаем метрики по кросс-валидации
def get_cv_roc_pr_rc(clf, skf, X, y):
    roc_auc = []
    pr_auc = []
    precision = []
    recall = []
    models = []
    for train, test in skf.split(X, y):
        models.append(clf.fit(X.iloc[train,:], y.iloc[train]))
        y_pred = clf.predict(X.iloc[test,:])
        y_pred_prob = clf.predict_proba(X.iloc[test,:])[:,1]
        roc_auc.append(metrics.roc_auc_score(y.iloc[test], y_pred_prob))
        pr_auc.append(metrics.average_precision_score(y.iloc[test], y_pred_prob))
        precision.append(metrics.precision_score(y.iloc[test], y_pred))
        recall.append(metrics.recall_score(y.iloc[test], y_pred))

    print ('ROC-AUC:', np.mean(roc_auc))
    print ('PR-AUC:', np.mean(pr_auc))
    print ('Precission:', np.mean(precision))
    print ('Recall:', np.mean(recall))
    
    return roc_auc, pr_auc, precision, recall, models

In [0]:
from sklearn.model_selection import StratifiedKFold

In [0]:
# разобьем рабочие данные для кросс-валидации, поделим на 9 подвыборок
skf = StratifiedKFold(n_splits=9, random_state=19)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [0]:
%%time

# Логистическая регрессия
clf = LogisticRegression(n_jobs=-1)
roc_auc, pr_auc, precision, recall, models = get_cv_roc_pr_rc(clf, skf, x_data, y_data)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

ROC-AUC: 0.5696737980286505
PR-AUC: 0.5553799490747788
Precission: 0.5544953811666338
Recall: 0.5865620419949792
CPU times: user 496 ms, sys: 855 ms, total: 1.35 s
Wall time: 14.2 s


In [0]:
%%time

# Случайный лес
clf = RandomForestClassifier(n_estimators = 100, n_jobs=-1)
roc_auc, pr_auc, precision, recall, models = get_cv_roc_pr_rc(clf, skf, x_data, y_data)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was pas

ROC-AUC: 0.6737413046335187
PR-AUC: 0.6482135366488394
Precission: 0.6281469906536132
Recall: 0.6354425144747726
CPU times: user 18 s, sys: 385 ms, total: 18.4 s
Wall time: 12 s


In [0]:
%%time

# Градиентный бустинг
clf = GradientBoostingClassifier(n_estimators = 30, random_state = 19)
roc_auc, pr_auc, precision, recall, models = get_cv_roc_pr_rc(clf, skf, x_data, y_data)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
 

ROC-AUC: 0.7153771974879031
PR-AUC: 0.7086459518297834
Precission: 0.6400421249054581
Recall: 0.6771545281732039
CPU times: user 13.2 s, sys: 12.4 ms, total: 13.2 s
Wall time: 13.3 s


Были обучены три модели: логистическая регрессия, случайный лес и градиентный бустинг. Все три модели показали примерно одинаковое качество, за отправные значения примем метрики, показанные Градиентным бустингом:

ROC-AUC: 0.716606066648193
Precission: 0.6460469878765588
Recall: 0.6843121036669424

In [0]:
# подгружаем тестовые данные для Kaggle
test_kaggle = pd.read_csv('orange_small_churn_test_data.csv', sep=',')

In [0]:
print (x_data.shape)
print (test_kaggle.shape)

(4460, 230)
(10000, 231)


In [0]:
# удаляем ненужную колонку ID из тестовых данных
del test_kaggle['ID']
#заменяем все пропуски в датасете на нули:
x_test_kaggle = test_kaggle.fillna(0)

In [0]:
# Encode first DataFrame 2 (where there is a combination of floats and strings)
x_test_kaggle = x_test_kaggle.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')
print("x_test_kaggle encoded")

x_test_kaggle encoded


In [0]:
# удаление неинформативных фич
# Неинформативными считаю фичи, в которых одно константное значение 
def dropUninformative(data):
    df = pd.DataFrame([])
    for i in range(data.shape[1]):
        if len(np.unique(data.iloc[:,i]))>1:
            df["Var"+str(i)] = data.iloc[:,i]
    return df

In [0]:
# удаляем неинформативные фичи
temp = dropUninformative(pd.DataFrame.append(x_data,x_test_kaggle))

In [0]:
clf = GradientBoostingClassifier(n_estimators = 30, random_state = 19).fit(x_data, y_data)
pred_answers = clf.predict_proba(x_test_kaggle)
answers = pd.DataFrame({'ID':range(0, len(pred_answers)), 'result':pred_answers[:,1]})
answers.to_csv('./answers_GradientBoostingClassifier1.csv', sep = ',', index=False)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Результат: https://drive.google.com/open?id=144hpnV_jlgoRRjdTcSlvpoWgot7tZcXu

87 dima ziaziulia 0.68595